In [ ]:
!pip install map_boxes==1.0.5
!pip install numpy==1.19.0
!pip install tqdm==4.65.0
!pip install pycocotools==2.0.6

In [1]:
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO

Couldn't import fast version of function compute_overlap, will use slow one. Check cython intallation


In [2]:
'''
    GT_JSON: valid set json file path
    현재 예시에서는 train.json을 임의로 valid set으로 사용
    PRED_CSV: 위의 valid set을 inference한 submission file
'''
# GT_JSON = '../../dataset/train.json'
GT_JSON = '/opt/ml/dataset/train.json'
# PRED_CSV = '../../sample_submission/train_sample.csv'
PRED_CSV = '/opt/ml/baseline/level2_objectdetection-cv-11/UniverseNet/work_dirs/gflv2_pvt_v2_b2_fpn_albu_1024_trash/submission_valid.csv'
LABEL_NAME = ["General trash", "Paper", "Paper pack", "Metal", 
              "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]
    
# load ground truth
with open(GT_JSON, 'r') as outfile:
    test_anno = (json.load(outfile))

# load prediction
pred_df = pd.read_csv(PRED_CSV)

In [3]:
'''
[
    [file_name label_index confidence_score x_min x_max y_min y_max], 
    [file_name label_index confidence_score x_min x_max y_min y_max],
    ,,,
    [file_name label_index confidence_score x_min x_max y_min y_max]
]

이는 아래의 mean_average_precision_for_boxes 함수에 들어갈 양식입니다.
new_pred가 위와 같은 format을 갖도록, submission.csv를 가공해봅시다.
'''
    
new_pred = []

file_names = pred_df['image_id'].values.tolist()
bboxes = pred_df['PredictionString'].values.tolist()
    
'''
create new_pred
'''

# check variable type
for i, bbox in enumerate(bboxes):
    if isinstance(bbox, float):
        print(f'{file_names[i]} empty box')

for file_name, bbox in tqdm(zip(file_names, bboxes)):
    boxes = np.array(str(bbox).strip().split(' '))

    # boxes - class ID confidence score xmin ymin xmax ymax
    if len(boxes) % 6 == 0:
        boxes = boxes.reshape(-1, 6)
    elif isinstance(bbox, float):
        print(f'{file_name} empty box')
        continue
    else:
        raise Exception('error', 'invalid box count')
    for box in boxes:
        new_pred.append([file_name, box[0], box[1], float(box[2]), float(box[4]), float(box[3]), float(box[5])])

4883it [00:01, 2626.93it/s]


In [4]:
'''
[
    [file_name 1, label_index, x_min, x_max, y_min, y_max], 
    [file_name 2, label_index, x_min, x_max, y_min, y_max],
    ,,,
    [file_name , label_index, x_min, x_max, y_min, y_max]
]

이는 아래의 mean_average_precision_for_boxes 함수에 들어갈 양식입니다.
gt가 위와 같은 format을 갖도록, COCO API를 활용하여 가공해봅시다.
참고: https://github.com/cocodataset/cocoapi
'''
    
gt = []

   
'''
coco.getImgIds(): return image id list
    
coco.loadImgs(image_id): return image_info
    
image_info['file_name']: return file name
   
coco.getAnnIds(imgIds=image_info['id']): return annotation id
    
coco.loadAnns(ann_ids): return annotation information list (annotation_info_list)
    
annotation_info_list[i]['bbox']: return i'th annotation [x_min, y_min, w, h]
    
annotation_info_list[i]['category_id']: return i'th annotation category
    
'''
coco = COCO(GT_JSON)

'''
create gt
: image id 별로 GT 값 추가
'''
for image_id in coco.getImgIds():
        
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
        
    file_name = image_info['file_name']
        
    for annotation in annotation_info_list:
        gt.append([file_name, annotation['category_id'],
                   float(annotation['bbox'][0]),
                   float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                   float(annotation['bbox'][1]),
                   (float(annotation['bbox'][1]) + float(annotation['bbox'][3]))])

loading annotations into memory...
Done (t=0.20s)
creating index...
index created!


In [5]:
'''
calculate mAP
'''

mean_ap, average_precisions = mean_average_precision_for_boxes(gt, new_pred, iou_threshold=0.5)

print(mean_ap)

Number of files in annotations: 4883
Number of files in predictions: 4883
Unique classes: 10
Detections length: 4883
Annotations length: 4883
0                              | 0.543839 |    3966
1                              | 0.670299 |    6352
2                              | 0.782903 |     897
3                              | 0.711771 |     936
4                              | 0.823532 |     982
5                              | 0.677763 |    2943
6                              | 0.746696 |    1263
7                              | 0.833467 |    5178
8                              | 0.939261 |     159
9                              | 0.804582 |     468
mAP: 0.753411
0.7534112969285742


In [6]:
'''
이번 미션에서는 train.json을 이용했지만, 실제 컴피티션에서는 validation set에 대한 모델의 성능을 측정, 트래킹하는 것이 중요합니다.
validation.json을 만들어, mAP를 계산, 모델의 성능을 측정해봅시다.

또한, 심화 미션은 라이브러리를 이용하지 않고, mean_average_precision_for_boxes 함수를 구현하는 것입니다.
도전해볼까요?
'''

'\n이번 미션에서는 train.json을 이용했지만, 실제 컴피티션에서는 validation set에 대한 모델의 성능을 측정, 트래킹하는 것이 중요합니다.\nvalidation.json을 만들어, mAP를 계산, 모델의 성능을 측정해봅시다.\n\n또한, 심화 미션은 라이브러리를 이용하지 않고, mean_average_precision_for_boxes 함수를 구현하는 것입니다.\n도전해볼까요?\n'

###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.
